In [2]:
import numpy as np
import pandas as pd

import scripts
import os
import subprocess
import importlib

In [3]:
importlib.reload(scripts)

<module 'scripts' from '/net/holy-nfsisilon/ifs/rc_labs/eddy_lab/users/lmerk/phage_groupII/scripts.py'>

In [11]:
data_dir = 'data/'

# Infernal subdir
infernal_dir = os.path.join(data_dir, 'infernal/')
g1_intron_hits = os.path.join(infernal_dir, 'g1_intron_IMGVR.tblout')
g2_intron_hits = os.path.join(infernal_dir, 'g2_intron_IMGVR.tblout')

# Genomes subdir
genomes_dir = os.path.join(data_dir, 'genomes/')
genbank_dir = os.path.join(genomes_dir, 'IMGVR/')
# actual_genomes = pd.read_csv(f'{genomes_dir}actual_IMGVR_genomes.txt', header=None)[0].unique()

# Window and defense
window_dir = os.path.join(genomes_dir, 'gII_intron_IMGVR_5kb_windows/')
bakta_dir = os.path.join(data_dir, 'IMGVR_bakta_output')
pharokka_dir = os.path.join(data_dir, "IMGVR_pharokka_output")
defense_dir = os.path.join(data_dir, 'IMGVR_defensefinder/IMGVR_bakta_defense')

# Script output subdir
script_output = os.path.join(data_dir, 'IMGVR_script_output/')
genbank_out_directory = os.path.join(script_output, "IMGVR_updated_genomes")

In [9]:
esl-sfetch -f 'IMGVR_UViG_3300015373_000925' -C

SyntaxError: invalid syntax (<ipython-input-9-fac716c94d82>, line 1)

In [ ]:
esl-sfetch -f /n/eddy_lab/data/IMGVR-2022-12-19_7/IMGVR_all_nucleotides.fna all_imgvr_hits.txt

In [6]:
preprocess = True

______

In [49]:
def extract_hit_df(filename, filt = True):
    initial = 0
    colspecs = []

    # Initialize an empty list to store the cleaned data
    
    cleaned_data = []

    # Read the file line by line, clean each line, and append to cleaned_data
    with open(filename, 'r') as infile:
        for line in infile:
            # Remove leading and trailing white spaces, split by any white space
            cleaned_line = line.strip().split()
            cleaned_data.append(cleaned_line)

    # Convert the cleaned data list into a DataFrame
    hit_df = pd.DataFrame(cleaned_data[2:-10])

    # Naming columns as per your data
    hit_df.columns = ['target_name', 'accession', 'query_name', 'accession_2', 'mdl',
                      'mdl_from', 'mdl_to', 'seq_from', 'seq_to', 'strand', 'trunc',
                      'pass', 'gc', 'bias', 'score', 'E-value', 'inc', 'description_of_target']

    return hit_df

In [66]:
all_genomes = [x.split('_')[2] for x in g2_df.target_name.unique()]
all_genomes_df = pd.DataFrame({'genomes': all_genomes})
all_genomes_df.to_csv('all_genomes_imgvr.csv', index = False)

In [72]:
all_genomes = [x.split('|')[0] for x in g2_df.target_name.unique()]
all_genomes_df = pd.DataFrame({'genomes': all_genomes})
all_genomes_df.to_csv('all_uvig_imgvr.csv', index = False, sep= ',')

In [73]:
g2_df.to_csv('whole_imgvr_g2.csv', index = False, sep= ',')

In [51]:
if preprocess:
    g1_df = extract_hit_df(g1_intron_hits)
    g2_df = extract_hit_df(g2_intron_hits)

#     g1_df = pd.merge(g1_hits, metadata, on='target_name', how='left')
#     g2_df = pd.merge(g2_hits, metadata, on='target_name', how='left')
    
    print('Dereplicating group I...')
    g1_hits_pass, g1_hits_fail = scripts.dereplicate_hits(g1_df)

    print('Saving group I...')
    # Save all the hits
    g1_df.sort_values('target_name').to_csv(f'{script_output}g1_df.csv', index=False)
    # Save the ones that didn't pass the filter
    g1_hits_fail.to_csv(f'{script_output}g1_hits_fail.csv', index=False)
    # Clean the passes, add the intronID, then save it
    g1_hits_pass = g1_hits_pass.sort_values(by=['target_name', 'seq_from'])
    g1_hits_pass['intronID'] = g1_hits_pass['target_name'] + '_I_' + g1_hits_pass['seq_from'].astype(str)
    column_order = ['intronID'] + [col for col in g1_hits_pass.columns if col != 'intronID']
    g1_hits_pass = g1_hits_pass[column_order]
    g1_hits_pass.to_csv(f'{script_output}g1_hits_pass.csv', index=False)
    
    
    print('Collapsing group II...')
    g2_hits_pass = scripts.collapse_hits(g2_df)
    
    print('Saving group II...')
    g2_hits_pass = g2_hits_pass.sort_values(by=['target_name', 'seq_from'])
    g2_hits_pass['intronID'] = g2_hits_pass['target_name'] + '_II_' + g2_hits_pass['seq_from'].astype(str)
    column_order = ['intronID'] + [col for col in g2_hits_pass.columns if col != 'intronID']
    g2_hits_pass = g2_hits_pass[column_order]
    g2_hits_pass.to_csv(f'{script_output}g2_hits_pass.csv', index=False)

    # Save all the hits
    g2_df = g2_df.sort_values(by=['target_name', 'seq_from'])
    g2_df['intronID'] = g2_df['target_name'] + '_II_' + g2_df['seq_from'].astype(str)
    column_order = ['intronID'] + [col for col in g2_df.columns if col != 'intronID']
    g2_df = g2_df[column_order]
    g2_df.to_csv(f'{script_output}g2_df.csv', index=False)
    print('Done!')
    
else:
    g1_df = pd.read_csv(f'{script_output}g1_df.csv')
    g2_df = pd.read_csv(f'{script_output}g2_df.csv')

    g1_hits_pass = pd.read_csv(f'{script_output}g1_hits_pass.csv')
    g2_hits_pass = pd.read_csv(f'{script_output}g2_hits_pass.csv')

Dereplicating group I...


KeyError: 'start'

In [8]:
# This will submit jobs to slurm to hmmscan the surrounding window
# The genomes should be indexed (i.e. contain a .idx file in the dir)
# If not, you can use esl-index.sh

if preprocess:
    scripts.hmmscan_window(g2_df, genbank_dir, window_dir)

In [9]:
# Note you need to make sure all your jobs from above have finished before running this
# You also need to run the following bash scripts before running this:
# pharokka.sh in pharokka env to generate annotations
# bakta.sh in bakta env to generate annotations
# defensefinder.sh in defensefinder env to get defense

if preprocess:
    all_defense_hits = scripts.combine_defense(g2_df, bakta_dir, defense_dir)
    all_hit_df = scripts.hit_to_genome_with_domtbl(g2_df, window_dir, genbank_dir)
    all_hit_df_derep = all_hit_df.loc[all_hit_df['genome'].isin(actual_genomes)]
    all_hit_df_derep.to_csv(f'{script_output}g2_genes_top3_5kb_with_locs.csv', index=False)
    scripts.combine_genbanks(all_hit_df_derep, pharokka_dir, bakta_dir, genbank_out_directory, actual_genomes)
    scripts.update_genbank(genbank_out_directory, all_hit_df_derep, g1_hits_pass, g2_df, all_defense_hits)
else:
    all_hit_df_derep =  pd.read_csv(f'{script_output}g2_genes_top3_5kb_with_locs.csv')